In [6]:
# hkust vislab code challenge
# This one is to implement a Year/Month HeatMap
# Level 1 Challenge
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mplcursors as mp
from matplotlib.widgets import Button
from matplotlib import colors

# first step to read all data from csv file
df = pd.read_csv('temperature_daily.csv')

# add two columns of data field 'year' and 'year-month'
df['year'] = pd.DatetimeIndex(df['date']).year
df['year_month'] = pd.to_datetime(df['date']).dt.to_period('M')
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.to_datetime(df['date']).dt.day
df1 = df.set_index(['year'])



# generate two pivot table to store the dict of data
table_min = pd.pivot_table(data=df1, index='month', columns='year', values='min_temperature', aggfunc=np.min)
table_max = pd.pivot_table(data=df1, index='month', columns='year', values='max_temperature', aggfunc=np.max)


index = ['January', 'Febuary', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
cols = table_min.columns

# generate hover text
def getHoverText():
    hovertext = list()
    for yi, yy in enumerate(table_min.index):
        hovertext.append(list())
        for xi, xx in enumerate(table_min.columns):
            value_min = (table_min.values[yi][xi]).astype(int)
            value_max = (table_max.values[yi][xi]).astype(int)
            hovertext[-1].append('Year: {}\n Month: {}\n MinValue: {}\n MaxValue: {}'.format(xx, yy, value_min, value_max))
    return hovertext

# define subplots
fig, ax = plt.subplots(figsize=(16, 8))

# define heatmap
cmap = colors.ListedColormap(["#5E4FA2","#3288BD","#66C2A5","#ABDDA4","#E6F598","#FFFFBF","#FEE08B","#FDAE61","#F46D43","#D53E4F","#9E0142"])
bounds = [0, 5, 10, 15, 20, 25, 30, 35, 40]
norm = colors.BoundaryNorm(bounds, cmap.N)

def heatmap (data, row_labels, col_labels, ax=None, **kwargs):
    if not ax:
        ax = plt.gca()

    # draw heatmap
    # make a color map of fixed colors
    im = ax.imshow(data, **kwargs)
    fig.colorbar(im,cmap=cmap, norm=norm, boundaries=bounds)

    # set the xticks and yticks of heatmap
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)


    # Turn spines off and create white grid.
    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    # horizontal axes labeling appear on top
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # set linewidth of heatmap
    ax.set_xticks(np.arange(data.shape[1] + 1) - .5, minor=True)
    ax.set_yticks(np.arange(data.shape[0] + 1) - .5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im

# switch function
class Index(object):
    def __init__(self, ax, **kwargs):
        self.ax = ax
        self.kwargs = kwargs

    def display_min(self, event):
        self.ax.imshow(table_min.values, **self.kwargs)
        plt.draw()

    def display_max(self, event):
        self.ax.imshow(table_max.values, **self.kwargs)
        plt.draw()

# draw heatmap
im = heatmap(table_min.values, index, cols, ax=ax,
                   cmap=cmap, norm=norm)

# set hovertext here
hover = getHoverText()
cursorText = mp.cursor(im, hover=True)
@cursorText.connect("add")
def _(sel):
    sel.annotation.get_bbox_patch().set(fc='w', alpha=0.8)
    sel.annotation.set_text(hover[sel.target.index[0]][sel.target.index[1]])
    sel.annotation.set_zorder(20)

# define button to switch
callback = Index(ax, cmap=cmap, norm=norm)
ax_min = plt.axes([0.35, 0.002, 0.1, 0.05])
ax_max = plt.axes([0.5, 0.002, 0.1, 0.05])
b_min = Button(ax_min, 'MinValue', color='#66C2A5', hovercolor='#3288BD')
b_min.on_clicked(callback.display_min)
b_max = Button(ax_max, 'MaxValue', color='#D53E4F', hovercolor='#9E0142')
b_max.on_clicked(callback.display_max)

#plt.tight_layout()
plt.show()
